In [2]:
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D

In [3]:
no_of_classes = 5
img_rows, img_cols = 48,48
batch_size = 8

train_data_dir = r'C:\Users\GAURAV\Desktop\sem\AI\aiproject\finalaiproject\dataset\train'
validation_data_dir = r'C:\Users\GAURAV\Desktop\sem\AI\aiproject\finalaiproject\dataset\validation'

In [4]:
train_data_gen = ImageDataGenerator(rescale=1./255,
                                    rotation_range=20,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    zoom_range=0.2,
                                    shear_range=0.2,
                                    horizontal_flip=True)

In [5]:
validation_data_gen = ImageDataGenerator(rescale=1./255)

In [6]:
train_generator = train_data_gen.flow_from_directory(train_data_dir,
                                                     target_size=(img_rows,img_cols),
                                                     color_mode='grayscale',
                                                     batch_size=batch_size,
                                                     class_mode='categorical',
                                                     shuffle = True)     

validation_generator = validation_data_gen.flow_from_directory(validation_data_dir,
                                                     target_size=(img_rows,img_cols),
                                                     color_mode='grayscale',
                                                     batch_size=batch_size,
                                                     class_mode='categorical',
                                                     shuffle = True)

Found 24283 images belonging to 5 classes.
Found 5937 images belonging to 5 classes.


In [7]:
model = Sequential()

# Block-1

model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-2 

model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-3

model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-4 

model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-5

model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block-6

model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block-7

model.add(Dense(no_of_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

In [8]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 32)        0         
__________

In [9]:
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint('Emotion_model.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          #restore_best_weights=True
                          )

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks = [earlystop,checkpoint,reduce_lr]

model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics=['accuracy'])

nb_train_samples = 24176
nb_validation_samples = 3006
epochs=25

history=model.fit_generator(
                train_generator,
                steps_per_epoch=nb_train_samples//batch_size,
                epochs=epochs,
                callbacks=callbacks,
                validation_data=validation_generator,
                validation_steps=nb_validation_samples//batch_size)

Epoch 1/25
3022/3022 [==============================] - 751s 248ms/step - loss: 1.7284 - acc: 0.2628 - val_loss: 1.5842 - val_acc: 0.2442

Epoch 00001: val_loss improved from inf to 1.58423, saving model to Emotion_model.h5
Epoch 2/25
3022/3022 [==============================] - 785s 260ms/step - loss: 1.5231 - acc: 0.3138 - val_loss: 1.3877 - val_acc: 0.4132

Epoch 00002: val_loss improved from 1.58423 to 1.38767, saving model to Emotion_model.h5
Epoch 3/25
3022/3022 [==============================] - 796s 264ms/step - loss: 1.3920 - acc: 0.4076 - val_loss: 1.1695 - val_acc: 0.5139

Epoch 00003: val_loss improved from 1.38767 to 1.16945, saving model to Emotion_model.h5
Epoch 4/25
3022/3022 [==============================] - 800s 265ms/step - loss: 1.2801 - acc: 0.4744 - val_loss: 1.0541 - val_acc: 0.5877

Epoch 00004: val_loss improved from 1.16945 to 1.05410, saving model to Emotion_model.h5
Epoch 5/25
3022/3022 [==============================] - 814s 269ms/step - loss: 1.2210 - acc